In [1]:
import torch.backends.cudnn as cudnn
import torch.utils.data
import torchvision.transforms as transforms
# from datasets import *
# from utils import *
# from nltk.translate.bleu_score import corpus_bleu
import torch.nn.functional as F
from tqdm import tqdm
import argparse
import json
import torchvision.models as models
# from nltk.translate.bleu_score import corpus_bleu
import pickle

import json, os

import time
from torch import nn
from torch.nn.utils.rnn import pack_padded_sequence

import torch.optim as optim

from models import *
from demo_func import *

In [2]:
import warnings
warnings.filterwarnings('ignore')

language1 = 'EN'
language2 = 'DE'
language3 = 'JP'

with open(os.path.join('{}.json'.format(language1)), 'r') as j:
    word_map1 = json.load(j)
with open(os.path.join('{}.json'.format(language2)), 'r') as j:
    word_map2 = json.load(j)
with open(os.path.join('{}.json'.format(language3)), 'r') as j:
    word_map3 = json.load(j) 
all_word_maps = {language1:word_map1, language2:word_map2, language3:word_map3}

rev_dict_en = {v:k for k,v in all_word_maps['EN'].items()}
rev_dict_de = {v:k for k,v in all_word_maps['DE'].items()}
rev_dict_jp = {v:k for k,v in all_word_maps['JP'].items()}
all_rev_dict = {'EN': rev_dict_en, 'DE':rev_dict_de, 'JP':rev_dict_jp}
 

# load model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
checkpoint = torch.load("LSTM_three_languages.pth.tar", map_location=str(device))


decoder = DecoderWithTripleAttention(attention_dim=512, embed_dim=300, decoder_dim=512, vocabs = all_word_maps, encoder_dim=2048, dropout=0.1)
decoder.load_state_dict(checkpoint['decoder'].state_dict())

decoder = decoder.to(device)
decoder.eval()
encoder = LF(14)
encoder.load_state_dict(checkpoint['encoder'].state_dict())
encoder = encoder.to(device)
encoder.eval()

LF(
  (resnet): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv2d(64, 256, k

In [3]:
class Arguments:
    def __init__(self):
        self.layer = 'conv13' 
        self.src = 'EN' # EN or DE or JP
        self.tgt = 'DE' # EN or DE or JP
        self.lr = 0.01 # 0.001 for normal, 0.01 for random
        self.iteration = 11 # 6 or 9 for normal, 11 for random, but this iteration varies depending on language pairs
        self.decoder_mode = 'lstm'
        self.type = 'random' # normal = image input, random = noise input
args = Arguments()

In [4]:
import imageio
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

single_noise = np.load('/p/visionlab/zy3cx/test_171.npy')
noise_img = torch.from_numpy(single_noise).view(3,224,224)

    
image_folder = '/localtmp/zy3cx/flickr30k-images'
img_name = '1007129816.jpg'
img = imageio.imread(image_folder + '/' + img_name)
img = np.array(Image.fromarray(img).resize((256, 256)))
img = img.transpose(2, 0, 1)
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
img_tensor  = torch.FloatTensor(img / 255.)
img_tensor = normalize(img_tensor)
img_tensor = img_tensor.unsqueeze(0)

if args.type == 'random':
    print('noise!')
    img_tensor = noise_img.unsqueeze(0)
    
print(img_tensor.shape)


noise!
torch.Size([1, 3, 224, 224])


In [5]:
def get_translation(args, sent, img_tensor, all_word_maps, all_rev_dict):
    max_len = 50
#     sent = 'A Boston Terrier is running on lush green grass in front of a white fence.'.lower()
    c = sent.split()
    sample_caps = torch.LongTensor([all_word_maps[args.src]['<start>']] + [all_word_maps[args.src].get(word, all_word_maps[args.src]['<unk>']) for word in c] + [
                        all_word_maps[args.src]['<end>']] + [all_word_maps[args.src]['<pad>']] * (max_len - len(c)))
    sample_caplen = torch.LongTensor([len(c)+2])

    ini_out, out = feedback_test(args, encoder, decoder, img_tensor, sample_caps, sample_caplen, all_word_maps)
    print('before: ', ' '.join([all_rev_dict[args.tgt][x] for x in ini_out[0]]))
    print('after: ', ' '.join([all_rev_dict[args.tgt][x] for x in out[0] if x!=all_word_maps[args.tgt]['<unk>']]))
    return

# sentences = ['A black and white dog is running in a grassy garden surrounded by a white fence.',\
#             'A Boston Terrier is running on lush green grass in front of a white fence.',\
#             'A black and white dog is running through the grass.',\
#             'A dog runs on the green grass near a wooden fence.',\
#             'A Boston terrier is running in the grass.']
# sentences = ['Women in store happily holding some food.',
#                 'A man is sharpening a knife.',
#                 'Three people sit in a cave.',
#                 'A little girl is walking on a balance beam.',
#                 'A blond holding hands with a guy in the sand.']
sentences = ['A girl sits in a chair and reads a book.']
for sent in sentences:
    sent = sent.lower()
    get_translation(args, sent, img_tensor, all_word_maps, all_rev_dict)

before:  ein <unk> mit <unk> .
after:  ein mädchen sitzt in einem auf dem boden .
